In [ ]:
import pickle
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from cdlib import algorithms
from tqdm import tqdm

from hp_nlp_graph.bookgraph import BookGraph

In [ ]:
book_number = 4
book = BookGraph(
    book_number, f"data/processed/{book_number}/interactions_by_chapter.pkl"
)

In [ ]:
book.plot_metric_over_chapters("pagerank")

In [ ]:
book.plot_centrality_diagram()